In [3]:
import pandas as pd
import numpy as np

In [4]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict
import re

pd.set_option('display.max_colwidth' , -1)

import sys
import re
import os
from pathlib import Path
from collections import namedtuple
import numpy as np

In [5]:
import xml.etree.ElementTree as ET
english = ET.parse('../data/xml_files/English_restaurants.xml')
root = english.getroot()
labeled_reviews = []
for review in root.findall("Review"):
    rid = review.get('rid')
    if review.find("sentences"):
        for sentence in review.find("sentences").findall('sentence'): 
            if sentence.find('Opinions'):
                #print(sentence[0].text)
                entry= {} ;   aspects = [] ;  polarities = []
                for opinion in sentence.find('Opinions').findall('Opinion'):
                    aspects.append(opinion.get('category'))
                    polarities.append(opinion.get('polarity'))
            
                entry['review_id'] = rid#.copy()
                entry["text"] = sentence[0].text
                entry["aspects"] , entry['polarities']  =  aspects , polarities
                labeled_reviews.append(entry)
            
eng_multi_aspects = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")

#eng_multi_aspects.to_csv('../data/English_restaurants.csv' , index = False)

there are 1708 reviews in this training set


In [6]:
df_complete_review = eng_multi_aspects.groupby(['review_id'])['text'].apply(lambda x: ' '.join(x) ).reset_index()

In [7]:
df_complete_review.tail(2)

,review_id,text
347,Z#7,"This place doesn't make any sense Just want to warn you all - don't waste your time and money. This place has totally weird decor, stairs going up with mirrored walls - I am surprised how no one yet broke their head or fall off the stairs - mirrored walls make you dizzy and delusional... This place is not inviting and the food is totally weird. The concept of japanese tapas is newly created and clearly doesn't work. The food they serve is not comforting, not appetizing and uncooked."
348,Z#9,"short and sweet – seating is great:it's romantic,cozy and private. The boths are not as small as some of the reviews make them out to look they're perfect for 2 people. The service was extremely fast and attentive(thanks to the service button on your table) but I barely understood 1 word when the waiter took our order. The food was ok and fair nothing to go crazy. Over all the looks of the place exceeds the actual meals. So what you really end up paying for is the restaurant not the food. Will prob. not return but it is a great dinning experience to try atleast once."


In [6]:
# #from neuralcoref import Coref
# import en_core_web_sm
# spacy = en_core_web_sm.load()

In [7]:
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
import neuralcoref

/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:

import spacy#en_core_web_sm
nlp = spacy.load('en_core_web_sm')#en_core_web_sm.load()

In [ ]:
neuralcoref.add_to_pipe(nlp)

def replace_pronouns(text):
    result = doc._.coref_resolved  
    return result 

In [ ]:
doc = nlp(u'My sister has a dog. She loves him.')

In [1]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


In [15]:
doc_complete = df_complete_review['text'].values.tolist()

In [16]:
doc_clean = [clean(doc).split() for doc in doc_complete]    

In [21]:
doc_clean[-1][0:5]

['short', 'sweet', '–', 'seating', 'greatits']

In [22]:
import gensim
from gensim import corpora

In [24]:
dictionary = corpora.Dictionary(doc_clean)


In [25]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [34]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=100)

In [36]:
print(ldamodel.print_topics(num_topics=5, num_words=10))

[(0, '0.017*"food" + 0.016*"great" + 0.012*"good" + 0.012*"service" + 0.011*"restaurant" + 0.011*"place" + 0.006*"wine" + 0.006*"price" + 0.005*"one" + 0.005*"best"'), (1, '0.017*"food" + 0.015*"service" + 0.010*"good" + 0.007*"it" + 0.006*"place" + 0.006*"get" + 0.006*"go" + 0.005*"worth" + 0.004*"atmosphere" + 0.004*"dinner"'), (2, '0.019*"food" + 0.017*"great" + 0.014*"service" + 0.012*"restaurant" + 0.011*"place" + 0.010*"sushi" + 0.009*"go" + 0.008*"time" + 0.008*"good" + 0.006*"like"'), (3, '0.020*"place" + 0.020*"food" + 0.016*"good" + 0.011*"great" + 0.010*"pizza" + 0.008*"best" + 0.008*"go" + 0.008*"service" + 0.006*"would" + 0.006*"back"'), (4, '0.022*"food" + 0.015*"place" + 0.012*"great" + 0.011*"good" + 0.008*"service" + 0.007*"time" + 0.007*"it" + 0.006*"like" + 0.006*"drink" + 0.006*"price"')]


In [39]:
#restaurant_data = pd.read_csv('/home/FRACTAL/swati.tiwari/kaggle/yelp/Unsupervised-Aspect-Extraction-master/datasets/restaurant_old/train.txt' , header = None , sep =' ')

In [58]:
restaurant_data = '/home/FRACTAL/swati.tiwari/kaggle/yelp/Unsupervised-Aspect-Extraction-master/datasets/restaurant_old/train.txt'

In [59]:
file = open(restaurant_data , 'r')  

In [60]:
rest_280k = file.readlines()

In [68]:
rest_280k[210]

'We waited over an hour and were eventually seated at folding tables with vinyl table cloths and folding chairs\n'

In [62]:
doc_clean = [clean(doc).split() for doc in rest_280k]    

In [69]:
doc_clean[210]

['waited',
 'hour',
 'eventually',
 'seated',
 'folding',
 'table',
 'vinyl',
 'table',
 'cloth',
 'folding',
 'chair']

In [70]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [71]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=300,)

In [73]:
print(ldamodel.print_topics(num_topics=5, num_words=20))

[(0, '0.022*"time" + 0.020*"u" + 0.018*"table" + 0.015*"waiter" + 0.015*"went" + 0.014*"night" + 0.013*"get" + 0.012*"dinner" + 0.011*"order" + 0.010*"wait" + 0.009*"friend" + 0.008*"even" + 0.008*"reservation" + 0.008*"minute" + 0.007*"came" + 0.007*"got" + 0.007*"two" + 0.007*"last" + 0.007*"pasta" + 0.007*"seated"'), (1, '0.025*"pizza" + 0.016*"steak" + 0.015*"fresh" + 0.014*"chicken" + 0.011*"sauce" + 0.011*"salad" + 0.011*"dish" + 0.009*"like" + 0.009*"best" + 0.009*"good" + 0.008*"delicious" + 0.008*"cheese" + 0.007*"try" + 0.007*"fish" + 0.007*"burger" + 0.006*"side" + 0.006*"meat" + 0.006*"also" + 0.006*"ordered" + 0.006*"disappointed"'), (2, '0.039*"food" + 0.027*"good" + 0.025*"menu" + 0.021*"really" + 0.019*"wine" + 0.012*"well" + 0.010*"portion" + 0.010*"everything" + 0.009*"dessert" + 0.009*"special" + 0.008*"tasty" + 0.008*"nothing" + 0.008*"price" + 0.008*"dish" + 0.007*"selection" + 0.006*"also" + 0.006*"list" + 0.006*"worth" + 0.006*"delicious" + 0.006*"diner"'), (3, '